<a href="https://colab.research.google.com/github/Pranav-d33/Soccer_finetuning/blob/main/finetune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine-Tuning Setup

This notebook contains the steps to fine-tune a pre-trained model on the soccer tactics dataset.

In [1]:
# Import Required Libraries
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset
import torch

# Check if GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


In [2]:
# Install required packages
!pip install gdown unsloth

In [3]:
# Download dataset from Google Drive
import os

# Create a directory to store the dataset
os.makedirs('data', exist_ok=True)

# Use gdown to download files from the shared Google Drive folder
!gdown --folder https://drive.google.com/drive/folders/1izgyjcd4k6V7txzYcgtx5-cDzTdfRAmN?usp=drive_link -O data

print("Dataset downloaded and saved to 'data' folder.")

Retrieving folder contents
Processing file 1c2RxrUsUIMtxxUeJ0EfTloHYIgMK2ORZ football_train.jsonl
Processing file 18dWOtzzmelGOPb785BpKobJX7lvDZtot football_val.jsonl
Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From (original): https://drive.google.com/uc?id=1c2RxrUsUIMtxxUeJ0EfTloHYIgMK2ORZ
From (redirected): https://drive.google.com/uc?id=1c2RxrUsUIMtxxUeJ0EfTloHYIgMK2ORZ&confirm=t&uuid=9c14b6e1-636e-41c5-925b-422151be89fa
To: /content/data/football_train.jsonl
100% 2.00G/2.00G [00:27<00:00, 72.4MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=18dWOtzzmelGOPb785BpKobJX7lvDZtot
From (redirected): https://drive.google.com/uc?id=18dWOtzzmelGOPb785BpKobJX7lvDZtot&confirm=t&uuid=8bd23097-6335-4672-bc26-7cee7ef1f652
To: /content/data/football_val.jsonl
100% 222M/222M [00:04<00:00, 50.1MB/s]
Download completed
Dataset downloaded and saved to 'data' folder.


In [4]:
# Load Dataset from downloaded files
dataset = load_dataset('json', data_files={'train': 'data/football_train.jsonl', 'validation': 'data/football_val.jsonl'})
print(dataset)

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output', 'metadata'],
        num_rows: 2813850
    })
    validation: Dataset({
        features: ['instruction', 'input', 'output', 'metadata'],
        num_rows: 312650
    })
})


In [14]:
# Define Model and Tokenizer
from unsloth import FastLanguageModel
model_name = 'unsloth/Phi-3-mini-4k-instruct' # Changed model name to a compatible one
# Load model and tokenizer (Unsloth automatically provides both)
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=2048,
    dtype=torch.float16,    # or None (autodetect)
    load_in_4bit=True,      # optional for memory efficiency
    # Removed LoRA configuration from here
)

# DO NOT call model.to(device)
# The model is already on GPU and quantized.

# Optional: switch mode depending on goal
# FastLanguageModel.for_inference(model)  # for text generation
# FastLanguageModel.for_training(model)  # for fine-tuning

# Test generation
inputs = tokenizer("Explain quantum entanglement in simple terms.", return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=100)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

==((====))==  Unsloth 2025.10.11: Fast Mistral patching. Transformers: 4.57.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


TypeError: MistralForCausalLM.__init__() got an unexpected keyword argument 'lora_alpha'

In [6]:
# Preprocess Data
def preprocess_function(examples):
    # The dataset has 'instruction', 'input', 'output', and 'metadata' columns.
    # We need to combine these into a single 'text' column for the model.
    # The prompt format for Phi-3-mini-4k-instruct is:
    # <|user|>
    # {instruction}
    # {input}
    # <|end|>
    # <|assistant|>
    # {output}

    processed_text = []
    for instruction, input_text, output_text in zip(examples['instruction'], examples['input'], examples['output']):
        prompt = f"<|user|>\n{instruction}\n{input_text}\n<|end|>\n<|assistant|>\n{output_text}"
        processed_text.append(prompt)

    return tokenizer(processed_text, truncation=True, padding='max_length', max_length=512)

tokenized_datasets = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/2813850 [00:00<?, ? examples/s]

Map:   0%|          | 0/312650 [00:00<?, ? examples/s]

In [12]:
import transformers
print(transformers.__version__)

from transformers import TrainingArguments
print(TrainingArguments.__module__)



4.57.1
transformers.training_args


In [13]:
# Enable fine-tuning mode (adds LoRA adapters)
FastLanguageModel.for_training(model)

# --- Define TrainingArguments ---
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch", # Changed evaluation strategy to epoch
    learning_rate=2e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=2,
    load_best_model_at_end=True,
    logging_dir="./logs",
    fp16=True,
    report_to="none",
    save_strategy="epoch", # Changed save strategy to epoch
)

# --- Define Trainer ---
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
)

trainer.train()

/tmp/ipython-input-1392343433.py:22: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


ValueError: You cannot perform fine-tuning on purely quantized models. Please attach trainable adapters on top of the quantized model to correctly perform fine-tuning. Please see: https://huggingface.co/docs/transformers/peft for more details

In [29]:
!pip install --upgrade transformers

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Save the Fine-Tuned Model to the specified Google Drive folder
model.save_pretrained('/content/drive/.shortcut-targets-by-id/1Cd0xb_JlPejeIvb_TPXmmNbhHQtTf0mO/fine_tuned_model')
tokenizer.save_pretrained('/content/drive/.shortcut-targets-by-id/1Cd0xb_JlPejeIvb_TPXmmNbhHQtTf0mO/fine_tuned_model')

In [ ]:
# Evaluate the Fine-Tuned Model
print("Evaluating the model...")
eval_results = trainer.evaluate(eval_dataset=tokenized_datasets['validation'])
print(f"Evaluation results: {eval_results}")